In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import seaborn
import gd_parameters as gd

from gd_features import Features
from gd_cnn import CnnModel

% matplotlib inline

/usr/local/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/usr/local/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/usr/local/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [2]:
plt.style.use("seaborn-white")

In [3]:
image_dir = "/home/nakamura/network_dissection/NetDissect/dissection/alexnet_imagenet_full_conv_384/html/image/"

In [4]:
data_dir = gd.data_dir
subject_list = gd.subject_list
roi_list = gd.roi_list
feature_file = gd.feature_file
feature_type = gd.feature_type
result_dir = gd.result_dir
feature_file = gd.feature_file
feature_type = gd.feature_type
result_dir = gd.result_dir
features = Features(os.path.join(data_dir, feature_file), feature_type)

In [5]:
# Feture selection settings
num_features = 1000

# CNN model settings
model_def = './data/cnn/alexnet_imagenet_full_conv/alexnet_imagenet_full_conv.prototxt'
model_param = './data/cnn/alexnet_imagenet_full_conv/alexnet_imagenet_full_conv.caffemodel'
cnn_layers = ('conv1', 'conv2', 'conv3', 'conv4', 'conv5', 'fc6-conv', 'fc7-conv', 'fc8-conv')

mean_image_file = './data/images/ilsvrc_2012_mean.npy'  # ImageNet Large Scale Visual Recognition Challenge 2012

# Misc settings
rand_seed = 2501

model = CnnModel(model_def, model_param, mean_image_file, batch_size=128, rand_seed=rand_seed)
net = model.net
feat_ind = model.feat_index

In [6]:
with open("./results_alex/UnitAccuracyMean.pkl", "rb") as f:
    acc_unit = pickle.load(f)

In [7]:
def get_channel(blob, n):
    """
    return index of channel wehere unit n in blob belongs to
    """
    ind = feat_ind[blob][n - 1]
    blob_shape = net.blobs[blob].data[0].shape
    resolution = blob_shape[1] * blob_shape[2]
    channel = ind // resolution
    
    return blob, channel

In [8]:
def get_ordered_iou(blob,channel, n=10):
    """return ordered_iou averaged in a layer as pandas.Series
    use ration of channels exceeding threshold
    """
    path= os.path.join("/home/nakamura/network_dissection/NetDissect/dissection/alexnet_imagenet_full_conv_384",
                       blob + "-iou.csv")
    iou = pd.read_csv(path, index_col=0)
    channel_iou = iou.iloc[channel]
    sorted_iou = channel_iou.sort_values(ascending=False)
    sorted_label = sorted_iou.index.tolist()[:n]
    label = ",".join(sorted_label)

    return label

In [9]:
def top_decodable(selection, n=10, worst=False):
    if selection in roi_list:
        acc_unit_temp = acc_unit.loc[roi]
    elif selection in features.layers:
        acc_unit_temp = acc_unit.loc[("VC", selection)]
        
    if worst:
        acc_sorted = acc_unit_temp.sort_values("accuracy", ascending=True)
        top_decodable = acc_sorted.index.values[:n]       
    else:
        acc_sorted = acc_unit_temp.sort_values("accuracy", ascending=False)
        top_decodable = acc_sorted.index.values[:n]
    
    return top_decodable

In [10]:
def top_decodable_channel(selection, n=10, worst=False):
    if selection in roi_list:
        top = top_decodable(selection, n, worst)
        top_channel = []
        for i in range(len(top)):
            top_channel.append(get_channel(top[i][0], top[i][1]))
            
    elif selection in features.layers:
        top = top_decodable(selection, n, worst)
        top_channel = []
        for i in range(len(top)):
            top_channel.append(get_channel(selection, top[i]))
            
    return top_channel

In [11]:
def get_channel_image(blob, channel):
    file_name = "{}-{:04d}.jpg".format(blob, channel)
    path_to_image = os.path.join(image_dir, file_name)
    return path_to_image

# make image table for top decodable concepts

In [ ]:
for roi in roi_list:
    top_channel = top_decodable_channel(roi)
    fig, axes = plt.subplots(10, 1, figsize=(20, 12))
    for ch, ax in zip(top_channel, axes):
        img = mpimg.imread(get_channel_image(*ch))
        ax.imshow(img)
        ax.set_axis_off()
    fig.savefig(os.path.join("./figures/", "top_decodable_images_{}.pdf".format(roi)), bbox_inches="tight")

In [ ]:
# for best channels
for feat in features.layers[:7]:
    top_channel = top_decodable_channel(feat, n=5)
    fig, axes = plt.subplots(5, 1, figsize=(20, 6))
    for ch, ax in zip(top_channel, axes):
        img = mpimg.imread(get_channel_image(*ch))
        ax.imshow(img)
        ax.set_axis_off()
    fig.savefig(os.path.join("./figures/", "top_decodable_images_{}.pdf".format(feat)), bbox_inches="tight")
    fig.savefig(os.path.join("./figures/png", "top_decodable_images_{}.png".format(feat)), bbox_inches="tight", dpi=300)

In [ ]:
# for worst channels
for feat in features.layers[:7]:
    top_channel = top_decodable_channel(feat, n=5, worst=True)
    fig, axes = plt.subplots(5, 1, figsize=(20, 6))
    for ch, ax in zip(top_channel, axes):
        img = mpimg.imread(get_channel_image(*ch))
        ax.imshow(img)
        ax.set_axis_off()
    fig.savefig(os.path.join("./figures/", "top_decodable_images_{}_worst.pdf".format(feat)), bbox_inches="tight")
    fig.savefig(os.path.join("./figures/png", "top_decodable_images_{}_worst.png".format(feat)), bbox_inches="tight",dpi=300)

# make table for top decodable concepts

In [ ]:
# make tab separated table
# top decodable concepts for each roi
n = 10
df = pd.DataFrame(index=np.arange(1,n + 1),columns=roi_list)

for roi in roi_list:
    top_channel = top_decodable_channel(roi, n)
    for i in range(n):
        # get ordered iou of top nth best decodable
        concepts = get_ordered_iou(*top_channel[i], n=n)
        df[roi].iloc[i]= concepts 

df.to_csv(os.path.join("./figures/" ,"decodable_concept_sbj_mean.csv"), sep="\t")

In [36]:
for roi in roi_list:
    top_channel = top_decodable_channel(roi, n)
    print_ch = []
    for ch in top_channel:
        print_ch.append("{0} {1}".format(ch[0], ch[1]))
    print("{}:".format(roi) + ", ".join(print_ch))

V1:conv3 37, conv3 30, conv3 346, conv3 37, conv4 11, conv1 72, conv4 69, conv2 80, conv4 83, conv1 48
V2:conv3 346, conv3 37, conv2 20, conv3 30, conv3 37, conv4 56, conv4 83, conv3 158, conv3 34, conv1 48
V3:conv2 20, conv3 37, conv3 30, conv4 83, fc7-conv 1991, conv3 346, conv3 37, conv3 116, fc7-conv 23, conv3 245
V4:fc7-conv 1991, fc6-conv 2362, conv3 37, fc7-conv 2219, fc7-conv 1632, fc7-conv 2599, fc6-conv 1532, fc7-conv 2292, fc6-conv 1532, conv3 30
LOC:fc7-conv 1991, fc6-conv 2362, conv2 56, fc6-conv 1953, fc7-conv 1438, fc7-conv 3743, fc7-conv 1632, fc6-conv 1532, fc6-conv 1532, fc7-conv 2647
FFA:fc7-conv 1438, fc7-conv 2292, fc7-conv 1991, fc6-conv 2362, fc7-conv 2647, fc6-conv 1532, fc6-conv 1532, fc7-conv 1632, fc6-conv 1953, conv2 109
PPA:fc7-conv 2292, fc7-conv 2599, fc7-conv 2779, fc7-conv 1438, fc7-conv 1991, fc7-conv 2284, fc7-conv 1632, fc7-conv 2219, conv2 114, fc7-conv 1481
LVC:conv3 37, conv3 346, conv3 30, conv3 37, conv4 365, conv4 83, conv2 20, conv4 56, conv1 

In [41]:
for feat in features.layers[:7]:
    print(feat)
    top_channel = top_decodable_channel(feat, n=5)
    print_ch = list(map(lambda x: str(x[1]), top_channel))
    
    print("best5:" + ", ".join(print_ch))
    top_channel = top_decodable_channel(feat, n=5, worst=True)
    print_ch = list(map(lambda x: str(x[1]), top_channel))
    print("worst5:" + ", ".join(print_ch))

conv1
best5:72, 48, 45, 64, 61
worst5:89, 36, 82, 71, 34
conv2
best5:20, 80, 132, 159, 109
worst5:61, 60, 117, 243, 185
conv3
best5:30, 37, 37, 346, 245
worst5:227, 39, 362, 74, 19
conv4
best5:56, 69, 11, 365, 214
worst5:367, 53, 236, 53, 229
conv5
best5:127, 50, 50, 239, 242
worst5:207, 211, 64, 196, 115
fc6-conv
best5:578, 1532, 1532, 2362, 3011
worst5:832, 3035, 848, 2698, 4059
fc7-conv
best5:1438, 1991, 1013, 2292, 2909
worst5:2123, 3384, 591, 2499, 2929


In [ ]:
# make tab separated table
# top decodable concepts for each layer
n = 10
df = pd.DataFrame(index=np.arange(1,n + 1),columns=features.layers[:7])

for feat in features.layers[:7]:
    top_channel = top_decodable_channel(feat, n)
    for i in range(n):
        # get ordered iou of top nth best decodable
        concepts = get_ordered_iou(*top_channel[i], n=n)
        df[feat].iloc[i]= concepts 

df.to_csv(os.path.join("./figures/" ,"decodable_concept_layers_sbj_mean.csv"), sep="\t")

In [ ]:
# for worst decodable channels
# make tab separated table
# top decodable concepts for each layer
n = 10
df = pd.DataFrame(index=np.arange(1,n + 1),columns=features.layers[:7])

for feat in features.layers[:7]:
    top_channel = top_decodable_channel(feat, n, worst=True)
    for i in range(n):
        # get ordered iou of top nth best decodable
        concepts = get_ordered_iou(*top_channel[i], n=n)
        df[feat].iloc[i]= concepts 

df.to_csv(os.path.join("./figures/" ,"decodable_concept_layers_sbj_mean_worst.csv"), sep="\t")

In [17]:
n = 10
ch_list = []

for roi in roi_list:
    top_channel = top_decodable_channel(roi, n)
    top_unit = top_decodable(roi, n)
    for i in range(n):
        layer, ch = top_channel[i]
        unit = top_unit[i][1]
        # get ordered iou of top nth best decodable
        concepts = get_ordered_iou(*top_channel[i], n=n)
        
        ch = {"layer": layer,
             "channel": ch,
             "unit": unit,
              "ROI":roi,
             "rank": i + 1,
             "concept": concepts}
        ch_list.append(ch)
columns = ["ROI","layer", "channel", "unit", "rank", "concept"]
df = pd.DataFrame(ch_list)
df = df[columns]
df.to_csv(os.path.join("./figures/" ,"decodable_channel_roi.csv"), sep="\t")

In [22]:
n = 10
ch_list = []

for feat in features.layers[:7]:
    top_channel = top_decodable_channel(feat, n)
    top_unit = top_decodable(feat, n)
    for i in range(n):
        layer, ch = top_channel[i]
        unit = top_unit[i]
        # get ordered iou of top nth best decodable
        concepts = get_ordered_iou(*top_channel[i], n=n)
        
        ch = {"layer": layer,
             "channel": ch,
             "unit": unit,
             "rank": i + 1,
             "concept": concepts}
        ch_list.append(ch)
columns = ["layer", "channel", "unit", "rank", "concept"]
df = pd.DataFrame(ch_list)
df = df[columns]
df.to_csv(os.path.join("./figures/" ,"decodable_channel_layer.csv"), sep="\t")

In [27]:
n = 10
ch_list = []

for feat in features.layers[:7]:
    top_channel = top_decodable_channel(feat, n, worst=True)
    top_unit = top_decodable(feat, n, worst=True)
    for i in range(n):
        layer, ch = top_channel[i]
        unit = top_unit[i]
        # get ordered iou of top nth best decodable
        concepts = get_ordered_iou(*top_channel[i], n=n)
        
        ch = {"layer": layer,
             "channel": ch,
             "unit": unit,
             "rank": i + 1,
             "concept": concepts}
        ch_list.append(ch)
columns = ["layer", "channel", "unit", "rank", "concept"]
df = pd.DataFrame(ch_list)
df = df[columns]
df.to_csv(os.path.join("./figures/" ,"decodable_channel_layer_worst.csv"), sep="\t")

In [28]:
df

,layer,channel,unit,rank,concept
0,conv1,89,695,1,"frilly,purple-c,dotted,polka-dotted,red-c,band..."
1,conv1,36,543,2,"banded,grid,striped,lined,windowpane,perforate..."
2,conv1,82,303,3,"banded,red-c,dotted,ball_pit-s,polka-dotted,sw..."
3,conv1,71,732,4,"chequered,striped,banded,zigzagged,dotted,polk..."
4,conv1,34,855,5,"chequered,zigzagged,perforated,striped,meshed,..."
5,conv1,56,474,6,"red-c,dotted,polka-dotted,frilly,pink-c,knitte..."
6,conv1,73,685,7,"bubbly,chequered,blue-c,banded,striped,lined,s..."
7,conv1,77,653,8,"banded,zigzagged,ball_pit-s,dotted,red-c,polka..."
8,conv1,94,645,9,"purple-c,frilly,dotted,banded,swirly,blue-c,pi..."
9,conv1,38,550,10,"grid,lined,woven,banded,car,building_facade-s,..."
